In [ ]:
import numpy as np
import scipy
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
import os
from utils import get_sample_freq, get_test_freq, get_stimulus, remove_outliers
from scipy.stats import ttest_ind, levene

mode = 'Frequency'
starting_point_types = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
window_size_types = [1,1.5,2,2.5,3,3.5,4]

stimulus_raw = scipy.io.loadmat('dat_stimulus.mat')
spk_sample_raw = scipy.io.loadmat('dat_spk_sample.mat')
spk_test_raw = scipy.io.loadmat('dat_spk_test.mat')

stimulus = stimulus_raw['stimulus']
spk_sample = spk_sample_raw['spk_sample']
spk_test = spk_test_raw['spk_test']

In [ ]:
result_DataFrame = pd.DataFrame(columns= ['Starting Point', 'Window Size'] + [f'Case {i+1}' for i in range(10)])

for starting_point, window_size in product(starting_point_types,window_size_types):
    row = {'Starting Point': [starting_point], 'Window Size': [window_size]}
    significance_level = 0.1
    ans = []
    for test_num in range(1, 11):
        test_spike = pd.read_csv(f'./test_{mode}/data_sp{starting_point}ws{window_size}/test_spike_case{test_num}.csv')
        candidate = []
        for i in range(1, 11):
            sample_spike = pd.read_csv(f'./sample_{mode}/data_sp{starting_point}ws{window_size}/sample_spike_neuron{i}.csv')
            p_list = []
            for degree in np.unique(stimulus[:,1]):
                test = remove_outliers(test_spike[test_spike['Degree'] == int(degree)][mode].values)
                sample = remove_outliers(sample_spike[sample_spike['Degree'] == int(degree)][mode].values)
                _, p_eqv = levene(test, sample)
                #if p_eqv < 0.05: ##Seems the all experiments fail to reject non-equnivariance hypothesis.
                # Should not completely trust the t-test result. Only take care it for EDA
                _, p = ttest_ind(test, sample)
                p_list.append(p >= significance_level)
            if sum(p_list) > 6:
                candidate.append(i)
        if len(candidate) == 0:
            ans.append(None)
        elif len(candidate) == 1:
            ans.append(candidate[0])
        elif len(candidate) > 1:
            ans.append(tuple(candidate))
    for i, val in enumerate(ans):
        row[f'Case {i+1}'] = [val]
    
    if None not in ans:
        result_DataFrame = pd.concat([result_DataFrame, pd.DataFrame(row)], axis=0)
result_DataFrame.to_csv('evenmoreAblation_noNone_noOutliers.csv', index = False)